In [2]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


In [13]:
loan = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\01_Loans\20220630\infocube_01_loan20220630.txt', sep = ",", header = 0, low_memory = False)


In [14]:
IF  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\2022\202206\working\Total_Impaired_Final_20220630.txt',sep = ",", header = 0, low_memory = False)


In [15]:
loan.columns = loan.columns.str.upper()
loan.columns = loan.columns.str.strip()

IF.columns = IF.columns.str.upper()
IF.columns = IF.columns.str.strip()


In [25]:
loan.M_PRODUCT_HIERARCHY_CD = loan.M_PRODUCT_HIERARCHY_CD.str.strip()
loan.M_PRODUCT_HIERARCHY_CD = loan.M_PRODUCT_HIERARCHY_CD.str.upper()

loan.M_FULL_WRITEOFF_FLG = loan.M_FULL_WRITEOFF_FLG.str.strip()
loan.M_FULL_WRITEOFF_FLG = loan.M_FULL_WRITEOFF_FLG.str.upper()

loan1 = loan.iloc[np.where((loan.M_PRODUCT_HIERARCHY_CD.isin(['H38','H39','U50','U1','L60','U4','L62','W58','H83','L66','U8','W57','L9']))&(loan.M_FULL_WRITEOFF_FLG=='N')&(loan.M_ACCT_STATUS_CD==1))]

In [26]:
loan1 = loan1[['AS_OF_DATE','M_ACCOUNT_NO','M_CUS_NO','LEVEL_6','M_SUB_MARKET_SEGMENT_CD','M_PRODUCT_HIERARCHY_CD','M_SOURCE_PROD_TYPE_CD','SOURCE_PROD_TYPE_DSC','M_MTHS_INSTALLMENT_IN_ARREARS','M_FINANCING_CONCEPT','M_BNM_BALANCE_SUM']]

IF = IF[['V_ORIGINAL_ACCOUNT_NUMBER','MKT_SUB_SEGMENT','OUTSTANDING_AFTER_EIR_UWI','V_D_CUST_FIRST_NAME']]

IF['IMPAIRED'] = 'Y'

In [27]:
loan1.M_ACCOUNT_NO = loan1.M_ACCOUNT_NO.map(str)
loan1.M_ACCOUNT_NO = loan1.M_ACCOUNT_NO.str.strip()
loan1.M_ACCOUNT_NO = loan1.M_ACCOUNT_NO.str.upper()

IF.V_ORIGINAL_ACCOUNT_NUMBER = IF.V_ORIGINAL_ACCOUNT_NUMBER.str.strip()
IF.V_ORIGINAL_ACCOUNT_NUMBER = IF.V_ORIGINAL_ACCOUNT_NUMBER.str.upper()

join_a = loan1.merge(IF.rename(columns= {'V_ORIGINAL_ACCOUNT_NUMBER':'M_ACCOUNT_NO'}), on='M_ACCOUNT_NO', how='left')

join_a.IMPAIRED.fillna('N',inplace=True)

In [28]:
join_a.M_BNM_BALANCE_SUM.fillna(0, inplace=True)
join_a.OUTSTANDING_AFTER_EIR_UWI.fillna(0, inplace=True)

group_a = join_a.groupby(['SOURCE_PROD_TYPE_DSC','M_PRODUCT_HIERARCHY_CD','IMPAIRED'])[['M_BNM_BALANCE_SUM','OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

In [31]:
writer = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\HYPR 2022 v1.xlsx', engine='xlsxwriter')

join_a.to_excel(writer, sheet_name='HYPR_Jun22_listing', index = False)
group_a.to_excel(writer, sheet_name='HYPR_Jun22', index = False)

writer.save()

# APC TRY

In [4]:
APC = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\13_Adverse Portfolio\2022\20220630\working\FINAL_APC_20220630(python).xlsx')

In [6]:
APC.M_Product_Hierarchy_Cd = APC.M_Product_Hierarchy_Cd.str.strip()
APC.M_Product_Hierarchy_Cd = APC.M_Product_Hierarchy_Cd.str.upper()

APC1 = APC.iloc[np.where(APC.M_Product_Hierarchy_Cd.isin(['H38','H39','U50','U1','L60','U4','L62','W58','H83','L66','U8','W57','L9']))]

In [7]:
APC1

,DATA_SOURCE,As_Of_Date,Level_6,M_Sub_Market_Segment_Cd,M_Product_Hierarchy_Cd,M_Source_Prod_Type_Cd,Source_Prod_Type_Dsc,M_Mths_Installment_In_Arrears,IMPAIRED_FLAG,F_NCIF_CUST,F_NCIF_ACCT,Security_Type_Cd,M_Financing_Concept,M_BNM_BALANCE,OUTSTANDING_AFTER_EIR_UWI,PRODUCT_GROUP_APC,PRODUCT_GROUP_APC2,OUTSTANDING_AFTER_EIR_UWI_NCIF,OS_AFTER_EIR_UWI_NCIF_ACCT,M_BNM_BALANCE_MIA23,M_BNM_BALANCE_MIA2,M_BNM_BALANCE_MIA3
1204,01. LOAN,30-JUN-2022,Other Personal Loan,3,W57,W57,EDU FIN-i CLEAN,0,0,0,0,0,B06,1.84e+06,0.00,12. Other Retail Financing,NaN,0.00,0.00,0.00,0.00,0.00
1205,01. LOAN,30-JUN-2022,Other Personal Loan,3,W58,W58,EDU FIN-i SECURED,0,0,0,0,0,B06,2.67e+05,0.00,12. Other Retail Financing,NaN,0.00,0.00,0.00,0.00,0.00
1469,01. LOAN,30-JUN-2022,Salary,3,L9,L9,Staff Salary Financing-i,0,0,0,0,0,A01,7.31e+06,0.00,11. Salary,08. SALARY,0.00,0.00,0.00,0.00,0.00
1470,01. LOAN,30-JUN-2022,Salary,3,L9,L9,Staff Salary Financing-i,0,0,0,0,0,A02,3.16e+06,0.00,11. Salary,08. SALARY,0.00,0.00,0.00,0.00,0.00
1471,01. LOAN,30-JUN-2022,Salary,3,L9,L9,Staff Salary Financing-i,0,0,0,0,0,A04,3.12e+06,0.00,11. Salary,08. SALARY,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,01. LOAN,30-JUN-2022,Unit Trust,3,U8,U8,ASB2-i Plus,1,0,0,0,0,B06,6.44e+06,0.00,09. Unit Trust,06. UNIT TRUST,0.00,0.00,0.00,0.00,0.00
1876,01. LOAN,30-JUN-2022,Unit Trust,3,U8,U8,ASB2-i Plus,2,0,0,0,0,B06,6.20e+05,0.00,09. Unit Trust,06. UNIT TRUST,0.00,0.00,620495.78,620495.78,0.00
1877,01. LOAN,30-JUN-2022,Unit Trust,3,U8,U8,ASB2-i Plus,3,0,0,0,0,B06,3.62e+05,0.00,09. Unit Trust,06. UNIT TRUST,0.00,0.00,362316.06,0.00,362316.06
1878,01. LOAN,30-JUN-2022,Unit Trust,3,U8,U8,ASB2-i Plus,4,Y,Y,Y,0,B06,2.04e+05,203801.17,09. Unit Trust,06. UNIT TRUST,203801.17,203801.17,0.00,0.00,0.00


In [9]:
APC1.M_BNM_BALANCE.fillna(0, inplace=True)
APC1.OUTSTANDING_AFTER_EIR_UWI.fillna(0, inplace=True)

group_apc = APC1.groupby(['Source_Prod_Type_Dsc','M_Product_Hierarchy_Cd','DATA_SOURCE'])[['M_BNM_BALANCE','OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

D:\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
